<a href="https://colab.research.google.com/github/ryamaguchi0111/gradio/blob/master/gradio_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NMISTを使ってgradio に入門
参考↓  
https://cpp-learning.com/gradio/

# 必要なモジュールのインストール

In [2]:
!pip install -q gradio

     |████████████████████████████████| 1.4MB 2.8MB/s 
     |████████████████████████████████| 215kB 17.6MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 2.7MB 15.4MB/s 
     |████████████████████████████████| 962kB 38.5MB/s 


# 必要なモジュールのインポート

In [12]:
import gradio as gr

import torch
import torch.nn as nn
import torch.nn.functional as f
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable

import requests
from PIL import Image

# ネットワークの構築


In [22]:
class EasyNet(nn.Module):
    def __init__(self):
        super(EasyNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        h = torch.sigmoid(self.fc1(x))
        h = self.fc2(h)
        h = f.log_softmax(h, dim=1)

        return h

In [9]:
net = EasyNet()
print(net)

EasyNet(
  (fc1): Linear(in_features=784, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=10, bias=True)
)


# dataset, dataloader の作成

In [6]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_datasets = datasets.MNIST(root='./data',
                                train=True,
                                download=True,
                                transform=transform)
test_datasets  = datasets.MNIST(root='./data',
                                train=False,
                                download=True,
                                transform=transform)

train_dataloader = DataLoader(train_datasets, 
                              batch_size=100,
                              shuffle=True,
                              num_workers=2)
test_dataloader  = DataLoader(test_datasets,
                              batch_size=100,
                              shuffle=False,
                              num_workers=2)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


学習

In [23]:

history = {
    'train_loss': [],
    'test_loss': [],
    'test_acc': []
}

net = EasyNet()


optimizer = torch.optim.Adam(params=net.parameters(), lr=0.001)

epoch = 20

for e in range(epoch):

     """ Training Part"""
     loss = None

     # 学習開始 (再開)
     net.train(True)  # 引数は省略可能

     for i, (data, target) in enumerate(train_dataloader):
        ########## 学習部分 ##########
        data = data.view(-1, 28*28)
        # data = Variable(data, requires_grad=True)

        optimizer.zero_grad()

        out = net(data)

        loss = f.nll_loss(out, target)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print(f'Training loss: {e+1} epoch ({(i+1)*128} / 60000 train. data). loss: {loss.item()}')

        history['train_loss'].append(loss)

     """ Test Part """
     # 学習のストップ
     net.eval()  # または net.train(False) でも良い
     test_loss = 0
     correct = 0

     with torch.no_grad():  # テスト部分では勾配は使わないのでこのように書く
         for data, target in test_dataloader:
             ########## テスト部分 ##########
             data = data.view(-1, 28*28)
             out = net(data)
             test_loss = f.nll_loss(out, target, reduction='sum').item()
             pred = out.argmax(dim=1, keepdim=True)
             correct += pred.eq(target.view_as(pred)).sum().item()
     test_loss /= 10000

     print(f'Test loss (avg): {test_loss}, Acciracy: {correct / 10000}')

     history['test_loss'].append(test_loss)
     history['test_acc'].append(correct / 10000)

Training loss: 1 epoch (128 / 60000 train. data). loss: 2.3826744556427
Training loss: 1 epoch (12928 / 60000 train. data). loss: 0.28015023469924927
Training loss: 1 epoch (25728 / 60000 train. data). loss: 0.31580689549446106
Training loss: 1 epoch (38528 / 60000 train. data). loss: 0.33446621894836426
Training loss: 1 epoch (51328 / 60000 train. data). loss: 0.3126375079154968
Training loss: 1 epoch (64128 / 60000 train. data). loss: 0.27933359146118164
Test loss (avg): 0.0032220958709716797, Acciracy: 0.9311
Training loss: 2 epoch (128 / 60000 train. data). loss: 0.18336699903011322
Training loss: 2 epoch (12928 / 60000 train. data). loss: 0.2375209927558899
Training loss: 2 epoch (25728 / 60000 train. data). loss: 0.3432992696762085
Training loss: 2 epoch (38528 / 60000 train. data). loss: 0.14960208535194397
Training loss: 2 epoch (51328 / 60000 train. data). loss: 0.09133890271186829
Training loss: 2 epoch (64128 / 60000 train. data). loss: 0.1195264458656311


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fd40f677b70>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1075, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fd40f677b70>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    sel

Test loss (avg): 0.0021036081314086915, Acciracy: 0.956
Training loss: 3 epoch (128 / 60000 train. data). loss: 0.23659424483776093
Training loss: 3 epoch (12928 / 60000 train. data). loss: 0.13355232775211334
Training loss: 3 epoch (25728 / 60000 train. data). loss: 0.07192034274339676
Training loss: 3 epoch (38528 / 60000 train. data). loss: 0.09911748766899109
Training loss: 3 epoch (51328 / 60000 train. data). loss: 0.07218758016824722
Training loss: 3 epoch (64128 / 60000 train. data). loss: 0.06590775400400162
Test loss (avg): 0.0014314044952392579, Acciracy: 0.9632
Training loss: 4 epoch (128 / 60000 train. data). loss: 0.11873501539230347
Training loss: 4 epoch (12928 / 60000 train. data). loss: 0.050801243633031845
Training loss: 4 epoch (25728 / 60000 train. data). loss: 0.09756823629140854
Training loss: 4 epoch (38528 / 60000 train. data). loss: 0.2131529450416565
Training loss: 4 epoch (51328 / 60000 train. data). loss: 0.07475445419549942
Training loss: 4 epoch (64128 / 6

# model save

In [24]:
PATH = './nmist.pth'
torch.save(net.state_dict(),PATH )

# model load

In [26]:
model = EasyNet()
model.load_state_dict(torch.load('./nmist.pth'))

<All keys matched successfully>

# 推論関数の作成

In [129]:
model.eval()
def predict(inp):
    # transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    inp = transform(inp).transpose(0,1)
    # print(inp.shape)
    inp = inp.reshape(-1, 28*28).float()
    # print(inp.shape)
    with torch.no_grad():
        # predict = model(inp).argmax(dim=1, keepdim=True)
        predict = model(inp)
        prediction = predict.numpy().squeeze().astype('float')
    return {str(i): np.exp(prediction[i]) for i in range(10)}

# gradio を使ってGUIの作成

In [133]:
sketchpad = gr.inputs.Sketchpad()
label = gr.outputs.Label(num_top_classes=3)
interface = gr.Interface(predict,
                         sketchpad,
                         label,
                         capture_session=True)
interface.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on External URL: https://56706.gradio.app
Interface loading below...


(<gradio.networking.serve_files_in_background.<locals>.HTTPServer at 0x7fd410a5b5f8>,
 'http://127.0.0.1:7872/',
 'https://56706.gradio.app')